# Kernel Programming

In this section we cover performance impact and consideration when writing kernel code SYCL:
- [Considerations for Selecting Work-group Size](#Considerations-for-Selecting-Work-group-Size)
- [Removing Conditional Checks](#Removing-Conditional-Checks)
- [Avoiding Register Spills](#Avoiding-Register-Spills)


## Considerations for Selecting Work-group Size
In SYCL you can select the work-group size for nd_range kernels. The size of work-group has important implications for utilization of the compute resources, vector lanes, and communication among the work-items. The work-items in the same work-group may have access to hardware resources like shared memory and hardware synchronization capabilities that will allow them to run and communicate more efficiently than work-items across work-groups. So in general you should pick the maximum work-group size supported by the accelerator. The maximum work-group size can be queried by the call `device::get_info<sycl::info::device::max_work_group_size>()`.

To illustrate the impact of the choice of work-group size, consider the following reduction kernel, which goes through a large vector to add all the elements in it. The function that runs the kernels takes in the work-group-size and sub-group-size as arguments, which lets you run experiments with different values. The performance difference can be seen from the timings reported when the kernel is called with different values for work-group size.

In [ ]:
%%writefile lab/wg_reduction.cpp
//==============================================================
// Copyright © Intel Corporation
//
// SPDX-License-Identifier: MIT
// =============================================================
#include <sycl/sycl.hpp>

// Summation of 10M 'one' values
constexpr size_t N = (10 * 1024 * 1024);

// Number of repetitions
constexpr int repetitions = 16;
// expected vlaue of sum
int sum_expected = N;

void init_data(sycl::queue &q, sycl::buffer<int> &buf, int data_size) {
  // initialize data on the device
  q.submit([&](auto &h) {
    sycl::accessor buf_acc(buf, h, sycl::write_only, sycl::no_init);
    h.parallel_for(data_size, [=](auto index) { buf_acc[index] = 1; });
  });
  q.wait();
}

void check_result(double elapsed, std::string msg, int sum) {
  if (sum == sum_expected)
    std::cout << "SUCCESS: Time is " << elapsed << "s" << msg << "\n";
  else
    std::cout << "ERROR: Expected " << sum_expected << " but got " << sum
              << "\n";
}

void reduction(sycl::queue &q, std::vector<int> &data, std::vector<int> &flush,
               int iter, int vec_size, int work_group_size) {
  const size_t data_size = data.size();
  const size_t flush_size = flush.size();
  int sum = 0;

  const sycl::property_list props = {sycl::property::buffer::use_host_ptr()};
  int num_work_items = data_size / work_group_size;
  sycl::buffer<int> buf(data.data(), data_size, props);
  sycl::buffer<int> flush_buf(flush.data(), flush_size, props);
  sycl::buffer<int> sum_buf(&sum, 1, props);

  init_data(q, buf, data_size);

  double elapsed = 0;
  for (int i = 0; i < iter; i++) {
    q.submit([&](auto &h) {
      sycl::accessor sum_acc(sum_buf, h, sycl::write_only, sycl::no_init);

      h.parallel_for(1, [=](auto index) { sum_acc[index] = 0; });
    });
    // flush the cache
    q.submit([&](auto &h) {
      sycl::accessor flush_acc(flush_buf, h, sycl::write_only, sycl::no_init);
      h.parallel_for(flush_size, [=](auto index) { flush_acc[index] = 1; });
    });

    auto start = std::chrono::high_resolution_clock::now().time_since_epoch().count();
    // reductionMapToHWVector main begin
    q.submit([&](auto &h) {
      sycl::accessor buf_acc(buf, h, sycl::read_only);
      sycl::local_accessor<int, 1> scratch(work_group_size, h);
      sycl::accessor sum_acc(sum_buf, h, sycl::write_only, sycl::no_init);

      h.parallel_for(
          sycl::nd_range<1>(num_work_items, work_group_size), [=
      ](sycl::nd_item<1> item) [[intel::reqd_sub_group_size(16)]] {
            auto v = sycl::atomic_ref<
                int, sycl::memory_order::relaxed,
                sycl::memory_scope::device,
                sycl::access::address_space::global_space>(sum_acc[0]);
            int sum = 0;
            int glob_id = item.get_global_id();
            int loc_id = item.get_local_id();
            for (int i = glob_id; i < data_size; i += num_work_items)
              sum += buf_acc[i];
            scratch[loc_id] = sum;

            for (int i = work_group_size / 2; i > 0; i >>= 1) {
	    sycl::group_barrier(item.get_group());
              if (loc_id < i)
                scratch[loc_id] += scratch[loc_id + i];
            }

            if (loc_id == 0)
              v.fetch_add(scratch[0]);
          });
    });
    q.wait();
    elapsed += (std::chrono::high_resolution_clock::now().time_since_epoch().count() - start) / 1e+9;
    sycl::host_accessor h_acc(sum_buf);
    sum = h_acc[0];
  }
  elapsed = elapsed / iter;
  std::string msg = " with work-groups=" + std::to_string(work_group_size);
  check_result(elapsed, msg, sum);
}

int main(int argc, char *argv[]) {

  sycl::queue q;
  std::cout << q.get_device().get_info<sycl::info::device::name>() << "\n";

  std::vector<int> data(N, 1);
  std::vector<int> extra(N, 1);

  int vec_size = 16;
  int work_group_size = vec_size;
  reduction(q, data, extra, 16, vec_size, work_group_size);
  work_group_size =
      q.get_device().get_info<sycl::info::device::max_work_group_size>();
  reduction(q, data, extra, 16, vec_size, work_group_size);

}



#### Build and Run
Select the cell below and click run ▶ to compile and execute the code:

In [ ]:
! ./q.sh run_wg_reduction.sh gen9

In situations where there are no barriers or atomics used, the work-group size will not impact the performance. To illustrate this, consider the following vec_copy kernel where there are no atomics or barriers.

In the code below, the above kernel is called with different work-group sizes. All the above calls to the kernel will have similar run times which indicates that there is no impact of work-group size on performance. The reason for this is that the threads created within a work-group and threads from different work-groups behave in a similar manner from the scheduling and resourcing point of view when there are no barriers or shared memory in the work-groups.



In [ ]:
%%writefile lab/wg_vec_copy.cpp
//==============================================================
// Copyright © Intel Corporation
//
// SPDX-License-Identifier: MIT
// =============================================================
#include <sycl/sycl.hpp>

// Copy of 32M 'one' values
constexpr size_t N = (32 * 1024 * 1024);

// Number of repetitions
constexpr int repetitions = 16;

void check_result(double elapsed, std::string msg, std::vector<int> &res) {
  bool ok = true;
  for (int i = 0; i < N; i++) {
    if (res[i] != 1) {
      ok = false;
      std::cout << "ERROR: Mismatch at " << i << "\n";
    }
  }
  if (ok)
    std::cout << "SUCCESS: Time " << msg << " = " << elapsed << "s\n";
}

void vec_copy(sycl::queue &q, std::vector<int> &src, std::vector<int> &dst,
              std::vector<int> &flush, int iter, int work_group_size) {
  const size_t data_size = src.size();
  const size_t flush_size = flush.size();
  int sum = 0;

  const sycl::property_list props = {sycl::property::buffer::use_host_ptr()};
  int num_work_items = data_size;
  double elapsed = 0;
  {
    sycl::buffer<int> src_buf(src.data(), data_size, props);
    sycl::buffer<int> dst_buf(dst.data(), data_size, props);
    sycl::buffer<int> flush_buf(flush.data(), flush_size, props);

    for (int i = 0; i < iter; i++) {
      // flush the cache
      q.submit([&](auto &h) {
        sycl::accessor flush_acc(flush_buf, h, sycl::write_only, sycl::no_init);
        h.parallel_for(flush_size, [=](auto index) { flush_acc[index] = 1; });
      });

      auto start = std::chrono::high_resolution_clock::now().time_since_epoch().count();
      q.submit([&](auto &h) {
        sycl::accessor src_acc(src_buf, h, sycl::read_only);
        sycl::accessor dst_acc(dst_buf, h, sycl::write_only, sycl::no_init);

        h.parallel_for(
            sycl::nd_range<1>(num_work_items, work_group_size), [=
        ](sycl::nd_item<1> item) [[intel::reqd_sub_group_size(16)]] {
              int glob_id = item.get_global_id();
              dst_acc[glob_id] = src_acc[glob_id];
            });
      });
      q.wait();
      elapsed += (std::chrono::high_resolution_clock::now().time_since_epoch().count() - start) / 1e+9;
    }
  }
  elapsed = elapsed / iter;
  std::string msg = "with work-group-size=" + std::to_string(work_group_size);
  check_result(elapsed, msg, dst);
} // vec_copy end

int main(int argc, char *argv[]) {

  sycl::queue q;
  std::cout << q.get_device().get_info<sycl::info::device::name>() << "\n";

  std::vector<int> src(N, 1);
  std::vector<int> dst(N, 0);
  std::vector<int> extra(N, 1);

  // call begin
  int vec_size = 16;
  int work_group_size = vec_size;
  vec_copy(q, src, dst, extra, 16, work_group_size);
  work_group_size = 2 * vec_size;
  vec_copy(q, src, dst, extra, 16, work_group_size);
  work_group_size = 4 * vec_size;
  vec_copy(q, src, dst, extra, 16, work_group_size);
  work_group_size = 8 * vec_size;
  vec_copy(q, src, dst, extra, 16, work_group_size);
  work_group_size = 16 * vec_size;
  vec_copy(q, src, dst, extra, 16, work_group_size);
  // call end
}


#### Build and Run
Select the cell below and click run ▶ to compile and execute the code:

In [ ]:
! ./q.sh run_wg_vec_copy.sh gen9

## Removing Conditional Checks
In Sub-groups, we learned that SIMD divergence can negatively affect performance. If all work items in a sub-group execute the same instruction, the SIMD lanes are maximally utilized. If one or more work items take a divergent path, then both paths have to be executed before they merge.

Divergence is caused by conditional checks, though not all conditional checks cause divergence. Some conditional checks, even when they do not cause SIMD divergence, can still be performance hazards. In general, removing conditional checks can help performance.

Look at the convolution example from Shared Local Memory:

In [ ]:
%%writefile lab/convolution_global.cpp
//==============================================================
// Copyright © Intel Corporation
//
// SPDX-License-Identifier: MIT
// =============================================================
#include <sycl/sycl.hpp>

int main() {
  constexpr size_t N = 8192 * 8192;
  constexpr size_t M = 257;

  std::vector<int> input(N);
  std::vector<int> output(N);
  std::vector<int> kernel(M);

  srand(2009);
  for (int i = 0; i < N; ++i) {
    input[i] = rand();
  }

  for (int i = 0; i < M; ++i) {
    kernel[i] = rand();
  }

  sycl::queue q{sycl::property::queue::enable_profiling{}};
  std::cout << "Device: " << q.get_device().get_info<sycl::info::device::name>() << "\n";

  {
    sycl::buffer<int> ibuf(input.data(), N);
    sycl::buffer<int> obuf(output.data(), N);
    sycl::buffer<int> kbuf(kernel.data(), M);

    auto e = q.submit([&](auto &h) {
      sycl::accessor iacc(ibuf, h, sycl::read_only);
      sycl::accessor oacc(obuf, h);
      sycl::accessor kacc(kbuf, h, sycl::read_only);

      h.parallel_for(sycl::nd_range<1>(N, 256), [=](sycl::nd_item<1> it) {
           int i = it.get_global_linear_id();
           int group = it.get_group()[0];
           int gSize = it.get_local_range()[0];

           int t = 0;

           if ((group == 0) || (group == N / gSize - 1)) {
             if (i < M / 2) {
               for (int j = M / 2 - i, k = 0; j < M; j++, k++) {
                 t += iacc[k] * kacc[j];
               }
             } else {
               if (i + M / 2 >= N) {
                 for (int j = 0, k = i - M / 2; j < M / 2 + N - i;
                      j++, k++) {
                   t += iacc[k] * kacc[j];
                 }
               } else {
                 for (int j = 0, k = i - M / 2; j < M; j++, k++) {
                   t += iacc[k] * kacc[j];
                 }
               }
             }
           } else {
             for (int j = 0, k = i - M / 2; j < M; j++, k++) {
               t += iacc[k] * kacc[j];
             }
           }

           oacc[i] = t;
         });
    });
    q.wait();

    size_t kernel_ns = (e.template get_profiling_info<sycl::info::event_profiling::command_end>() - e.template get_profiling_info<sycl::info::event_profiling::command_start>());
    std::cout << "Kernel Execution Time Average: total = " << kernel_ns * 1e-6 << " msec\n";
  }

  return 0;
}


#### Build and Run
Select the cell below and click run ▶ to compile and execute the code:

In [ ]:
! ./q.sh run_convolution_global.sh gen9

### Padding Buffers to Remove Conditional Checks

The nested if-then-else conditional checks are necessary to take care of the first and last 128 elements in the input so indexing will not run out of bounds. If we pad enough 0s before and after the input array, these conditional checks can be safely removed:

In [ ]:
%%writefile lab/convolution_global_conditionals.cpp
//==============================================================
// Copyright © Intel Corporation
//
// SPDX-License-Identifier: MIT
// =============================================================
#include <sycl/sycl.hpp>

int main() {
  constexpr size_t N = 8192 * 8192;
  constexpr size_t M = 257;

  sycl::queue q{sycl::property::queue::enable_profiling{}};
  std::cout << "Device: " << q.get_device().get_info<sycl::info::device::name>() << "\n";

  std::vector<int> input(N + M / 2 + M / 2);
  std::vector<int> output(N);
  std::vector<int> kernel(M);

  srand(2009);
  for (int i = M / 2; i < N + M / 2; ++i) {
    input[i] = rand();
  }

  for (int i = 0; i < M / 2; ++i) {
    input[i] = 0;
    input[i + N + M / 2] = 0;
  }

  for (int i = 0; i < M; ++i) {
    kernel[i] = rand();
  }

  {
    sycl::buffer<int> ibuf(input.data(), N + M / 2 + M / 2);
    sycl::buffer<int> obuf(output.data(), N);
    sycl::buffer<int> kbuf(kernel.data(), M);

    auto e = q.submit([&](auto &h) {
      sycl::accessor iacc(ibuf, h, sycl::read_only);
      sycl::accessor oacc(obuf, h);
      sycl::accessor kacc(kbuf, h, sycl::read_only);

      h.parallel_for(sycl::nd_range<1>(N, 256), [=](sycl::nd_item<1> it) {
           int i = it.get_global_linear_id();
           int t = 0;

           for (int j = 0; j < M; j++) {
             t += iacc[i + j] * kacc[j];
           }

           oacc[i] = t;
         });
    });
    q.wait();

    size_t kernel_ns = (e.template get_profiling_info<sycl::info::event_profiling::command_end>() - e.template get_profiling_info<sycl::info::event_profiling::command_start>());
    std::cout << "Kernel Execution Time Average: total = " << kernel_ns * 1e-6 << " msec\n";
  }

  return 0;
}


#### Build and Run
Select the cell below and click run ▶ to compile and execute the code:

In [ ]:
! ./q.sh run_convolution_global_conditionals.sh gen9

### Replacing Conditional Checks with Relational Functions
Another way to remove conditional checks is to replace them with relational functions, especially built-in relational functions. It is strongly recommended to use a built-in function if one is available. SYCL provides a rich set of built-in relational functions like `select()`, `min()`, `max()`. In many cases you can use these functions to replace conditional checks and achieve better performance.

Consider the convolution example again. The if-then-else conditional checks can be replaced with built-in functions `min()` and `max()`.

In [ ]:
%%writefile lab/convolution_global_conditionals_minmax.cpp
//==============================================================
// Copyright © Intel Corporation
//
// SPDX-License-Identifier: MIT
// =============================================================
#include <sycl/sycl.hpp>

int main() {
  constexpr size_t N = 8192 * 8192;
  constexpr size_t M = 257;

  std::vector<int> input(N);
  std::vector<int> output(N);
  std::vector<int> kernel(M);

  srand(2009);
  for (int i = 0; i < N; ++i) {
    input[i] = rand();
  }

  for (int i = 0; i < M; ++i) {
    kernel[i] = rand();
  }

  sycl::queue q{sycl::property::queue::enable_profiling{}};
  std::cout << "Device: " << q.get_device().get_info<sycl::info::device::name>() << "\n";

  {
    sycl::buffer<int> ibuf(input.data(), N);
    sycl::buffer<int> obuf(output.data(), N);
    sycl::buffer<int> kbuf(kernel.data(), M);

    auto e = q.submit([&](auto &h) {
      sycl::accessor iacc(ibuf, h, sycl::read_only);
      sycl::accessor oacc(obuf, h);
      sycl::accessor kacc(kbuf, h, sycl::read_only);

      h.parallel_for(sycl::nd_range<1>(N, 256), [=](sycl::nd_item<1> it) {
           int i = it.get_global_linear_id();
           int t = 0;
           int startj = sycl::max<int>(M / 2 - i, 0);
           int endj = sycl::min<int>(M / 2 + N - i, M);
           int startk = sycl::max<int>(i - M / 2, 0);
           for (int j = startj, k = startk; j < endj; j++, k++) {
             t += iacc[k] * kacc[j];
           }
           oacc[i] = t;
         });
    });
    q.wait();

    size_t kernel_ns = (e.template get_profiling_info<sycl::info::event_profiling::command_end>() - e.template get_profiling_info<sycl::info::event_profiling::command_start>());
    std::cout << "Kernel Execution Time Average: total = " << kernel_ns * 1e-6 << " msec\n";

  }

  return 0;
}


#### Build and Run
Select the cell below and click run ▶ to compile and execute the code:

In [ ]:
! ./q.sh run_convolution_global_conditionals_minmax.sh gen9

## Avoiding Register Spills
### Registers and Performance
It is well known that the register is the fastest storage in the memory hierarchy. Keeping data in registers as long as possible is critical to performance. On the other hand, register space is limited and much smaller than memory space. The current generation of Intel® GPUs, for example, has 128 general-purpose registers, each 32 bytes wide by default for each EU thread. Though the compiler aims to assign as many variables to registers as possible, the limited number of registers can be allocated only to a small set of variables at any point during execution. A given register can hold different variables at different times because different sets of variables are needed at different times. If there are not enough registers to hold all the variables, register can spill, or some variables currently in the registers can be moved to memory to make room for other variables.

In SYCL, the compiler allocates registers to private variables in work items. Multiple work items in a sub-group are packed into one EU thread. By default, the compiler uses register pressure as one of the heuristics to choose SIMD width or sub-group size. High register pressures can result in smaller sub-group size (for example 8 instead of 16) if a sub-group size is not explicitly requested. It can also cause register spilling or cause certain variables not to be promoted to registers.

The hardware may not be fully utilized if sub-group size or SIMD width is not the maximum the hardware supports. Register spilling can cause significant performance degradation, especially when spills occur inside hot loops. When variables are not promoted to registers, accesses to these variables incur significant increase of memory traffic.

Though the compiler uses intelligent algorithms to avoid or minimize register spills, optimizations by developers can help the compiler to do a better job and often make a big performance difference.
### Optimization Techniques
The following techniques can reduce register pressure:
- Keep live ranges of private variables as short as possible.
  Though the compiler schedules instructions and optimizes the distances, in some cases moving the loading and using the same variable closer or removing certain dependencies in the source can help the compiler do a better job.
- Avoid excessive loop unrolling.
  Loop unrolling exposes opportunities for instruction scheduling optimization by the compiler and thus can improve performance. However, temporary variables introduced by unrolling may increase pressure on register allocation and cause register spilling. It is always a good idea to compare the performance with and without loop unrolling and different times of unrolls to decide if a loop should be unrolled or how many times to unroll it.
- Prefer USM pointers.
  A buffer accessor takes more space than a USM pointer. If you can choose between USM pointers and buffer accessors, choose USM pointers.
- Recompute cheap-to-compute values on-demand that otherwise would be held in registers for a long time.
- Avoid big arrays or large structures, or break an array of big structures into multiple arrays of small structures.
  For example, an array of `sycl::float4`:
  `sycl::float4 v[8];`
  can be broken into 4 arrays of float:
  `float x[8]; float y[8]; float z[8]; float w[8];`
  All or part of the 4 arrays of float have a better chance to be allocated in registers than the array of `sycl::float4`.
- Break a large loop into multiple small loops to reduce the number of simultaneously live variables.
- Choose smaller data types if possible.
- Do not declare private variables as volatile.
- Share registers in a sub-group.
- Use shared local memory.

## Resources

- [Intel GPU Optimization Guide](https://www.intel.com/content/www/us/en/develop/documentation/oneapi-gpu-optimization-guide/top.html) - Up to date resources for Intel GPU Optimization
- [SYCL Specification](https://registry.khronos.org/SYCL/specs/sycl-2020/pdf/sycl-2020.pdf) - Latest Specification document for reference
- [SYCL Essentials Training](https://github.com/oneapi-src/oneAPI-samples/tree/master/DirectProgramming/C%2B%2BSYCL/Jupyter/oneapi-essentials-training) - Learn basics of C++ SYCL Programming